# Tutorial #4: Enable recurrent materialization and run batch inference

_Managed feature store is in private preview, which is subject to the [Supplemental Terms of Use for Microsoft Azure Previews](https://azure.microsoft.com/en-us/support/legal/preview-supplemental-terms/)_

In this tutorial series you will experience how features seamlessly integrates all the phases of ML lifecycle: Prototyping features, training and operationalizing.

In the previous part of the tutorial you learnt to experiment with features, train the model and register the model along with the feature-retrieval spec. In this tutorial you will learn how to run batch inference for the registered model.

You will perform the following:
- Enable recurrent materialization for the `transactions` feature set
- Run batch inference pipeline on the registered model

# Prerequisites
1. Please ensure you have executed the previous parts of this tutorial series

# Setup

#### (updated)  Configure Azure ML spark notebook

1. Running the tutorial: You can either create a new notebook, and execute the instructions in this document step by step or open the existing notebook named `4. Enable recurrent materialization and run batch inference`, and run it. The notebooks are available in `featurestore_sample/notebooks` directory. You can select from `sdk_only` or `sdk_and_cli`. You may keep this document open and refer to it for additional explanation and documentation links.
1. In the "Compute" dropdown in the top nav, select "AzureML Spark Compute". 
1. Click on "configure session" in bottom nav -> click on "upload conda file" -> select the file azureml-examples/sdk/python/featurestore-sample/project/env/conda.yml from your local machine; Also increase the session time out (idle time) if you want to avoid running the prerequisites frequently

#### Start spark session

In [1]:
# run this cell to start the spark session (any code block will start the session ). This can take around 10 mins.
print("start spark session")

StatementMeta(02c82d68-70da-4f10-89b1-7d661232d9b7, 324, 6, Finished, Available)

start spark session


#### Setup root directory for the samples

In [ ]:
import os

# please update the dir to ./Users/{your-alias} (or any custom directory you uploaded the samples to).
# You can find the name from the directory structure inm the left nav
root_dir = "./Users/<your user alias>/featurestore_sample"

if os.path.isdir(root_dir):
    print("The folder exists.")
else:
    print("The folder does not exist. Please create or fix the path")

#### (new for sdk/cki track) Setup CLI

1. Install azure ml cli extention
1. Authenticate
1. Set the default subscription

In [ ]:
!az extension add --name ml

In [ ]:
!az login

In [ ]:
import os

subscription_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]

!az account set -s $subscription_id

#### Initialize the project workspace CRUD client
This is the current workspace where you will be running the tutorial notebook from

In [2]:
### Initialize the MLClient of this project workspace
import os
from azure.ai.ml import MLClient
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

project_ws_sub_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]
project_ws_rg = os.environ["AZUREML_ARM_RESOURCEGROUP"]
project_ws_name = os.environ["AZUREML_ARM_WORKSPACE_NAME"]

# connect to the project workspace
ws_client = MLClient(
    AzureMLOnBehalfOfCredential(), project_ws_sub_id, project_ws_rg, project_ws_name
)

StatementMeta(02c82d68-70da-4f10-89b1-7d661232d9b7, 324, 7, Finished, Available)

#### Initialize the feature store variables
Ensure you update the `featurestore_name` to reflect what you created in part 1 of this tutorial

In [ ]:
featurestore_name = "my-featurestore"  # use the same name from part #1 of the tutorial
featurestore_subscription_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]
featurestore_resource_group_name = os.environ["AZUREML_ARM_RESOURCEGROUP"]

#### Initialize the feature store core sdk client

In [ ]:
# feature store client
from azureml.featurestore import FeatureStoreClient
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

featurestore = FeatureStoreClient(
    credential=AzureMLOnBehalfOfCredential(),
    subscription_id=featurestore_subscription_id,
    resource_group_name=featurestore_resource_group_name,
    name=featurestore_name,
)

## Step 1: Enable recurrent materialization on the `transactions` featureset

In part 2 of this tutorial you enabled materialization and performed backfill on the transactions feature set. Backfill is an ondemand one-time operation to compute and store feature values in the materialization store. However when you want to perform inference of the model in production, you might want to keep the materilization store upto date by setting up recurrent materialization jobs. These jobs run on user defined schedule
The recurrent job schedule works in the following way: 
- A window is defined by the interval and frequency. E.g., interval = 3 and frequency = Hour define a 3-hour window
- The first window starts at the start_time defined in the RecurenceTrigger, and so on.
- The first recurrent job will be submitted at the begining of the next window after the update time.
- Later recurrent jobs will be submitted at every window after the first job.

As explained in the previous parts of the tutorials, once data is materialized (backfill/recurrent materialization), feature retrieval will use the materialized data by default.

In [ ]:
feature_set_schedule_yaml = (
    root_dir
    + "/featurestore/featuresets/transactions/featureset_asset_offline_enabled_with_schedule.yaml"
)

!az ml feature-set update --file $feature_set_schedule_yaml --resource-group $featurestore_resource_group_name --workspace-name $featurestore_name

#### Track status of the recurrent materialization jobs in the feature store studio UI
This job will every three hours. 

__Action__:

1. Feel free to execute the next step for now (batch inference).
1. In three hours check the recurrent job status via the UI

## Step 2: Run the batch-inference pipeline

In this step you will manually trigger the batch inference pipeline. In a production scenario, this could be trigerred by a ci/cd pipeline based on model registration/approval.

The batch-inference has the following steps:

1. Feature retrieval step: This use the same built-in feature retrieval component that we used in the training pipeline in the part 3 of the tutorial. Incase of training pipeline, we provided feature retreival spec as an input to the component. However in case of batch inference we will pass the registered model as the input and the component will look for feature retrieval spec in the model artifact. Another difference is that in case of training, the observation data had the target variable, however incase of batch inference it will not be present. The feature retrieval step will join the observation data with the features and output the data for batch inference.
1. Batch inference: This step uses the batch inference input data from previous step, runs inference on the model and outputs the data by appending the predicted value.

__Note:__ In this example we use a job for batch inference. You can also use Azure ML's batch endpoints.

In [ ]:
# set the batch inference pipeline path
batch_inference_pipeline_path = (
    root_dir + "/project/fraud_model/pipelines/batch_inference_pipeline.yaml"
)

!az ml job create --file $batch_inference_pipeline_path --resource-group $project_ws_rg --workspace-name $project_ws_name

#### Inspect the batch inference output data
1. In the pipeline view, double click on `inference_step` -> in `outputs` card, copy the `Data` field. It will be something like `azureml_995abbc2-3171-461e-8214-c3c5d17ede83_output_data_data_with_prediction:1`. 
1. Paste it in the below cell with name and version separately (notice that the last character is the version, separated by a `:`).
1. You will see the `predict_is_fraud` column generated by the batch inference pipeline

Explanation: Since we did not provide a `name` and `version` in the `outputs` of the `inference_step` in the batch inference pipeline (`/project/fraud_mode/pipelines/batch_inference_pipeline.yaml`), the system created an untracked data asset with a guid as name and version as 1. In the next cell we will be getting the data path from the asset and displaying it.

In [4]:
# inf_data_output = ws_client.data.get(name="azureml_1c106662-aa5e-4354-b5f9-57c1b0fdb3a7_output_data_data_with_prediction", version="1")
inf_data_output = ws_client.data.get(
    name="azureml_0a7417c8-409a-4536-a069-4ea23a08ebfe_output_data_data_with_prediction",
    version="1",
)
inf_output_df = spark.read.parquet(inf_data_output.path)
display(inf_output_df.head(5))

StatementMeta(02c82d68-70da-4f10-89b1-7d661232d9b7, 324, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, e0a8aa54-ece6-49e3-ae90-2b4559c41ecc)

## Cleanup
If you created a resource group for the tutorial, you can delete the resource group to delete all the resources associated with this tutorial.

Otherwise, you can delete the resources individually:

* Delete the feature store: Go to the resource group in the azure portal, select the feature store and delete it
* Follow the instructions [here](https://review.learn.microsoft.com/en-us/azure/active-directory/managed-identities-azure-resources/how-manage-user-assigned-managed-identities?pivots=identity-mi-methods-azp&view=azureml-api-2#delete-a-user-assigned-managed-identity) to delete the user assigned managed identity
* Delete the offline store (storage account): Go to the resource group in the azure portal, select the storage you created and delete it